# D2-NET RESULTADOS (demo colab)

Drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Probar si se montó correctamente

In [9]:
!ls /content/drive/MyDrive/pruebas-pfc-2025/aachen_dataset/day/pair_01/

img1.png  img2.png


In [10]:
# 1. Instalar dependencias
!pip install opencv-python matplotlib numpy scipy tqdm

In [11]:
# 2. Clonar repositorio, y cambiar el nombre a la carpeta a d2net para que pueda ser leido

!git clone https://github.com/mihaidusmanu/d2-net.git
!mv d2-net d2net
%cd d2net
!mkdir -p models
!wget https://dusmanu.com/files/d2-net/d2_tf.pth -O models/d2_tf.pth

Cloning into 'd2-net'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 223 (delta 51), reused 43 (delta 43), pack-reused 155 (from 1)
Receiving objects: 100% (223/223), 2.35 MiB | 6.16 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/content/d2net/d2net
--2025-06-19 03:09:52--  https://dusmanu.com/files/d2-net/d2_tf.pth
Resolving dusmanu.com (dusmanu.com)... 142.132.238.25
Connecting to dusmanu.com (dusmanu.com)|142.132.238.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30545768 (29M) [application/octet-stream]
Saving to: ‘models/d2_tf.pth’

models/d2_tf.pth    100%[===================>]  29.13M  12.0MB/s    in 2.4s    

2025-06-19 03:09:56 (12.0 MB/s) - ‘models/d2_tf.pth’ saved [30545768/30545768]



In [12]:
# 3. Cargar librerías y preparar modelo
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.spatial.distance import cdist
from scipy.stats import entropy

import torch
from d2net.lib.model_test import D2Net
from d2net.lib.utils import preprocess_image
from d2net.lib.pyramid import process_multiscale

device = torch.device("cpu")
model = D2Net(model_file='models/d2_tf.pth', use_relu=True, use_cuda=False).to(device).eval()

# Funciones auxiliares
def extract_d2net_features(image):
    image = image.astype('float32') / 255.
    image = preprocess_image(image)  # esto devuelve numpy
    image = torch.from_numpy(image).unsqueeze(0)  # <- convertir a Tensor y agregar dimensión batch
    #if torch.cuda.is_available():
       # image = image.to(device)

    with torch.no_grad():
        keypoints, scores, descriptors = process_multiscale(image, model)

    # Seleccionar los Top-N keypoints por score
    #if len(scores) > top_n:
    #    idx = np.argsort(scores)[::-1][:top_n]
    #    keypoints = keypoints[idx]
    #    descriptors = descriptors[idx]

    return keypoints, descriptors

def match_features(desc1, desc2):
    distances = cdist(desc1, desc2, metric='euclidean')
    indices = np.argmin(distances, axis=1)
    min_distances = distances[np.arange(distances.shape[0]), indices]
    matches = np.stack((np.arange(len(indices)), indices), axis=1)
    return matches, min_distances

def spatial_entropy(keypoints, image_shape, grid_size=4):
    H, W = image_shape[:2]
    heatmap = np.zeros((grid_size, grid_size))
    for kp in keypoints:
        x, y = kp[:2]
        col = min(int(x / W * grid_size), grid_size - 1)
        row = min(int(y / H * grid_size), grid_size - 1)
        heatmap[row, col] += 1
    prob = heatmap.flatten()
    prob = prob / (prob.sum() + 1e-8)
    return entropy(prob)

def draw_matches(img1, kp1, img2, kp2, matches, save_path):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    new_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
    new_img[:h1, :w1] = img1
    new_img[:h2, w1:] = img2
    for match in matches:
        pt1 = tuple(np.round(kp1[match[0], :2]).astype(int))
        pt2 = tuple(np.round(kp2[match[1], :2]).astype(int) + np.array([w1, 0]))
        cv2.line(new_img, pt1, pt2, (0, 255, 0), 1)
    cv2.imwrite(save_path, new_img)


In [13]:
# 4. Procesar todas las imágenes del conjunto DAY

# Ruta base en Drive
base_path = '/content/drive/MyDrive/pruebas-pfc-2025/aachen_dataset/day'
result_dir = '/content/d2net_day_results'
os.makedirs(result_dir, exist_ok=True)

csv_data = []
top_n=500
# Iterar sobre cada par
for folder in tqdm(sorted(os.listdir(base_path))):
    pair_path = os.path.join(base_path, folder)
    img1_path = os.path.join(pair_path, 'img1.png')
    img2_path = os.path.join(pair_path, 'img2.png')

    if os.path.exists(img1_path) and os.path.exists(img2_path):
        img1 = cv2.imread(img1_path)
        img2 = cv2.imread(img2_path)

        kp1, desc1 = extract_d2net_features(img1)
        kp2, desc2 = extract_d2net_features(img2)

        matches, distances = match_features(desc1, desc2)

        # Métricas
        num_matches = len(matches)
        efficiency = num_matches / (min(len(desc1), len(desc2)) + 1e-8)
        avg_distance = float(np.mean(distances))

        kp1 = np.array(kp1)
        matched_kp1 = kp1[matches[:, 0]]

        spatial_ent = spatial_entropy(matched_kp1, img1.shape)

        output_img_path = os.path.join(result_dir, f'{folder}_matches.png')
        draw_matches(img1, kp1, img2, kp2, matches, output_img_path)

        csv_data.append({
            'pair': folder,
            'num_matches': num_matches,
            'efficiency': round(efficiency, 4),
            'avg_distance': round(avg_distance, 4),
            'spatial_entropy': round(spatial_ent, 4)
        })

# Guardar CSV
df = pd.DataFrame(csv_data)
csv_path = os.path.join(result_dir, 'd2net_day_results.csv')
df.to_csv(csv_path, index=False)

print(f"Proceso completado. Resultados guardados en:\n{csv_path}")


100%|██████████| 5/5 [28:39<00:00, 343.96s/it]

Proceso completado. Resultados guardados en:
/content/d2net_day_results/d2net_day_results.csv


In [14]:
!cat /proc/meminfo | grep Mem


MemTotal:       13289424 kB
MemFree:         6535876 kB
MemAvailable:   11101016 kB
